In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [4]:
df_all = pd.read_csv('CleanedData.csv', index_col=0)
df_all.head()

,Season,ThreePointers,ThreePointersAttempted,ThreePointersMissed,ThreePointersMadePct
Name,,,,,
A.J. Hammons,2017.0,5,10,5,0.500000
A.J. Lawson,2023.0,10,25,15,0.400000
A.J. Lawson,2024.0,12,36,24,0.333333
A.J. Price,2010.0,60,174,114,0.344828
A.J. Price,2011.0,48,165,117,0.290909


In [5]:
# historical 3 point contest results
df_historical = pd.read_csv('3PtHistoricalResults.csv')

# fill NA with 0s
df_historical = df_historical.fillna(0)
df_historical.head()

,name,made,att,dewmade,dewatt
0,Seth Curry,13,25,0.0,0.0
1,Stephen Curry,173,225,3.0,4.0
2,Buddy Hield,91,125,0.0,8.0
3,Damian Lillard,45,75,3.0,4.0
4,Kyle Korver,14,25,0.0,0.0
